In [1]:
import pandas as pd
import glob, os

In [2]:
# read TAZ-SD number crosswalk
taz_f = pd.read_csv('M:/Data/GIS layers/TM1_taz/bayarea_rtaz1454_rev1_WGS84.csv',
                    usecols = ['TAZ1454', 'SUPERD'])
taz_f.drop_duplicates(inplace=True)

# read SD number-name crosswalk 
sd_name = pd.read_csv(r'Z:\bayarea_urbansim\data\superdistricts.csv',
                      usecols=['number', 'name'])
sd_name.drop_duplicates(inplace=True)

# merge the two
TAZ_SD = taz_f.merge(sd_name, left_on='SUPERD', right_on='number', how='left')
TAZ_SD = TAZ_SD[['TAZ1454', 'SUPERD', 'name']]
TAZ_SD.rename(columns = {'TAZ1454': 'TAZ', 'name': 'SD_NAME'}, inplace=True)

display(TAZ_SD)

,TAZ,SUPERD,SD_NAME
0,1005,19,Berkeley Eville
1,999,19,Berkeley Eville
2,998,19,Berkeley Eville
3,1000,19,Berkeley Eville
4,1001,19,Berkeley Eville
...,...,...,...
1449,1136,22,Lamorinda WC
1450,1145,22,Lamorinda WC
1451,1146,22,Lamorinda WC
1452,1201,24,East Contra Costa


In [6]:
runs = pd.read_csv('taz_comparison_key.csv')
runs.set_index('output_file', inplace=True)
runs

,run_id,scenario,version,year
output_file,,,,
run267_taz_summaries_2015,run267,NP,v01,2015
run267_taz_summaries_2035,run267,NP,v01,2035
run267_taz_summaries_2050,run267,NP,v01,2050
run264_taz_summaries_2015,run264,P,v01,2015
run264_taz_summaries_2035,run264,P,v01,2035
run264_taz_summaries_2050,run264,P,v01,2050


In [ ]:
runs = pd.DataFrame()

In [7]:
for run_output in runs.index:
    print(run_output)
    
    df = pd.read_csv('taz_summary_tables\\' +run_output + '.csv', usecols=['TAZ', 'TOTHH', 'TOTEMP'])
    print(runs['version'][run_output])
    print(runs['year'][run_output])
    print(runs['scenario'][run_output])
    suffix = runs['version'][run_output] + '_' + str(runs['year'][run_output]) + '_' + runs['scenario'][run_output]
    df.rename(columns = {'TOTHH': 'TOTHH_' + suffix,
                         'TOTEMP': 'TOTEMP_' + suffix}, inplace=True)
    display(df.head())
    
    TAZ_SD = TAZ_SD.merge(df, on='TAZ', how='left')
    print(TAZ_SD.shape)

run267_taz_summaries_2015
v01
2015
NP


,TAZ,TOTEMP_v01_2015_NP,TOTHH_v01_2015_NP
0,1,18883,54.0
1,2,29033,135.0
2,3,3342,266.0
3,4,20341,181.0
4,5,16608,623.0


(1454, 5)
run267_taz_summaries_2035
v01
2035
NP


,TAZ,TOTEMP_v01_2035_NP,TOTHH_v01_2035_NP
0,1,19956.0,44.0
1,2,26959.0,137.0
2,3,3873.0,269.0
3,4,22685.0,206.0
4,5,17830.0,643.0


(1454, 7)
run267_taz_summaries_2050
v01
2050
NP


...

(1454, 11)
run264_taz_summaries_2035
v01
2035
P


,TAZ,TOTEMP_v01_2035_P,TOTHH_v01_2035_P
0,1,19960.0,52.0
1,2,27001.0,140.0
2,3,3904.0,272.0
3,4,22647.0,203.0
4,5,17808.0,652.0


(1454, 13)
run264_taz_summaries_2050
v01
2050
P


,TAZ,TOTEMP_v01_2050_P,TOTHH_v01_2050_P
0,1,23814.0,56.0
1,2,31921.0,141.0
2,3,4797.0,271.0
3,4,27148.0,208.0
4,5,23076.0,654.0


(1454, 15)


In [8]:
for col in list(TAZ_SD):
    if col not in ['TAZ', 'SUPERD', 'SD_NAME']:
        TAZ_SD[col].fillna(0, inplace=True)
print(list(TAZ_SD))

['TAZ', 'SUPERD', 'SD_NAME', 'TOTEMP_v01_2015_NP', 'TOTHH_v01_2015_NP', 'TOTEMP_v01_2035_NP', 'TOTHH_v01_2035_NP', 'TOTEMP_v01_2050_NP', 'TOTHH_v01_2050_NP', 'TOTEMP_v01_2015_P', 'TOTHH_v01_2015_P', 'TOTEMP_v01_2035_P', 'TOTHH_v01_2035_P', 'TOTEMP_v01_2050_P', 'TOTHH_v01_2050_P']


In [10]:
TAZ_SD['hh_growth_2035_v01_NP'] = TAZ_SD['TOTHH_v01_2035_NP'] - TAZ_SD['TOTHH_v01_2015_NP']
TAZ_SD['hh_growth_2050_v01_NP'] = TAZ_SD['TOTHH_v01_2050_NP'] - TAZ_SD['TOTHH_v01_2015_NP']

TAZ_SD['hh_growth_2035_v01_P'] = TAZ_SD['TOTHH_v01_2035_P'] - TAZ_SD['TOTHH_v01_2015_P']
TAZ_SD['hh_growth_2050_v01_P'] = TAZ_SD['TOTHH_v01_2050_P'] - TAZ_SD['TOTHH_v01_2015_P']

TAZ_SD['emp_growth_2035_v01_NP'] = TAZ_SD['TOTEMP_v01_2035_NP'] - TAZ_SD['TOTEMP_v01_2015_NP']
TAZ_SD['emp_growth_2050_v01_NP'] = TAZ_SD['TOTEMP_v01_2050_NP'] - TAZ_SD['TOTEMP_v01_2015_NP']

TAZ_SD['emp_growth_2035_v01_P'] = TAZ_SD['TOTEMP_v01_2035_P'] - TAZ_SD['TOTEMP_v01_2015_P']
TAZ_SD['emp_growth_2050_v01_P'] = TAZ_SD['TOTEMP_v01_2050_P'] - TAZ_SD['TOTEMP_v01_2015_P']

In [11]:
TAZ_SD.to_csv('taz_comparison.csv', index=False)

In [12]:
TAZ_SD

,TAZ,SUPERD,SD_NAME,TOTEMP_v01_2015_NP,TOTHH_v01_2015_NP,TOTEMP_v01_2035_NP,TOTHH_v01_2035_NP,TOTEMP_v01_2050_NP,TOTHH_v01_2050_NP,TOTEMP_v01_2015_P,...,TOTEMP_v01_2050_P,TOTHH_v01_2050_P,hh_growth_2035_v01_NP,hh_growth_2050_v01_NP,hh_growth_2035_v01_P,hh_growth_2050_v01_P,emp_growth_2035_v01_NP,emp_growth_2050_v01_NP,emp_growth_2035_v01_P,emp_growth_2050_v01_P
0,1005,19,Berkeley Eville,3120,1385.0,2759.0,1428.0,2638.0,1554.0,3134,...,2657.0,1500.0,43.0,169.0,80.0,122.0,-361.0,-482.0,-370.0,-477.0
1,999,19,Berkeley Eville,4819,852.0,5498.0,882.0,5975.0,907.0,4863,...,6009.0,916.0,30.0,55.0,21.0,56.0,679.0,1156.0,628.0,1146.0
2,998,19,Berkeley Eville,1693,2415.0,1635.0,3030.0,1684.0,3398.0,1684,...,1791.0,3115.0,615.0,983.0,615.0,724.0,-58.0,-9.0,-15.0,107.0
3,1000,19,Berkeley Eville,1111,1731.0,1069.0,1792.0,1123.0,1889.0,1136,...,1157.0,1881.0,61.0,158.0,91.0,145.0,-42.0,12.0,-91.0,21.0
4,1001,19,Berkeley Eville,701,1518.0,799.0,1586.0,856.0,1654.0,695,...,908.0,1650.0,68.0,136.0,68.0,149.0,98.0,155.0,92.0,213.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1449,1136,22,Lamorinda WC,2152,2516.0,2288.0,3568.0,2350.0,3681.0,2150,...,2364.0,3194.0,1052.0,1165.0,182.0,694.0,136.0,198.0,174.0,214.0
1450,1145,22,Lamorinda WC,18785,3534.0,19230.0,5931.0,19718.0,6686.0,18888,...,19780.0,6756.0,2397.0,3152.0,2646.0,3229.0,445.0,933.0,214.0,892.0
1451,1146,22,Lamorinda WC,11315,1868.0,14481.0,2053.0,14817.0,2355.0,11303,...,14542.0,2419.0,185.0,487.0,348.0,555.0,3166.0,3502.0,3125.0,3239.0
1452,1201,24,East Contra Costa,255,1130.0,282.0,1190.0,308.0,1188.0,256,...,314.0,1182.0,60.0,58.0,30.0,27.0,27.0,53.0,43.0,58.0


In [13]:
growth_cols = [col for col in TAZ_SD.columns if 'growth' in col]
TAZ_SD[['SD_NAME'] + growth_cols].groupby(['SD_NAME']).sum().reset_index()

,SD_NAME,hh_growth_2035_v01_NP,hh_growth_2050_v01_NP,hh_growth_2035_v01_P,hh_growth_2050_v01_P,emp_growth_2035_v01_NP,emp_growth_2050_v01_NP,emp_growth_2035_v01_P,emp_growth_2050_v01_P
0,Berkeley Eville,17972.0,31008.0,17603.0,28083.0,15749.0,21110.0,15045.0,22653.0
1,Central Marin,2738.0,6817.0,4429.0,7762.0,2329.0,5068.0,2298.0,4666.0
2,Daly City Millbrae,37551.0,68432.0,42349.0,72842.0,39352.0,55380.0,41812.0,50796.0
3,East Contra Costa,23189.0,36500.0,26296.0,39013.0,25595.0,39334.0,25968.0,38192.0
4,East San Jose,44748.0,89604.0,41573.0,84654.0,10305.0,29737.0,10334.0,30001.0
5,Fremont,27752.0,47242.0,25276.0,48897.0,49973.0,72533.0,50438.0,72749.0
6,Golden Triangle,138075.0,236038.0,151138.0,236013.0,159518.0,210873.0,156834.0,208916.0
7,Lamorinda WC,11762.0,18792.0,9427.0,19526.0,13234.0,15652.0,13136.0,16298.0
8,Martinez Concord,18665.0,35353.0,19578.0,34869.0,40891.0,85117.0,42850.0,86004.0
9,Napa City and S,3937.0,4352.0,3786.0,4190.0,9497.0,18825.0,9567.0,19203.0
